# scatter_visualization_v2
****
##説明

緯度・経度データから地図上にプロットし, 自動DLするプログラム
****
##ユーザへ

左側（デフォルト）にあるファイルタブから予め保安範囲や点火点, その他のデータをアップロードする.

ファイルは一行目の緯度, 経度に対応する列に**"longitude", "latitude"** ないし**"経度", "緯度"**と書かれていること.  
座標が1つの場合は点として, そうでない場合はポリゴンとして解釈される.

具体的な落下位置が書かれたcsvファイルは実行時に纏めてアップロードする.  
ここでは**"longitude", "latitude"** ないし**"経度", "緯度"**の文字が含まれた行を自動抽出する．

また, 結果はすべてZIPファイルに圧縮されてDLされる.

##下2つのセルを初めに必ず実行してください
plotly画像出力用ライブラリ・日本語フォントインストール＋自作クラス定義

忘れたら "ランタイム" -> "セッションを再起動" でOK.
それでも不調なら, "ランタイムを接続解除して削除" の上再接続.

折りたたむと２つのセルをいっきに実行できるので便利です.

In [1]:
# コンソール非表示
%%capture
# plotlyエクスポート用ライブラリ
!pip install -U kaleido
# 日本語フォントをダウンロード
!apt-get -y install fonts-ipafont-gothic
# キャッシュを削除
!rm /root/.cache/matplotlib/fontlist-v300.json # 消すべきcache
# (cf : https://qiita.com/siraasagi/items/3836cedede350280ec42)

In [30]:
import plotly.express as px       # 地図
import plotly.graph_objects as go # 地図
import geopandas as gpd     # 緯度・経度処理
from shapely.geometry import Point, Polygon, LineString # gpdの座標
import pandas as pd         # csvファイル
import numpy as np
from pathlib import Path    # パス
import shutil               # ディレクトリ削除
import os
import time                 # タイムスタンプ用
from google.colab import files # ファイルアップロード, ダウンロード

# 分散表示クラス
class scatter_map:
    def __init__(self):
        self.launch_site_dfs = [] # 射場に関するDataFrame
        self.fig = None
        self.fig_height = 450; self.fig_width = 600
        self.launch_site_centroid = None # 射場の重心
        self.zoom_lv = 15 # NSE 陸の場合．海なら12

    # ズームレベル調整
    # NSE陸は15, NSE海は12 が丁度いい感じ
    def set_zoom_level(self, lv):
        self.zoom_lv = lv

    # 点の大きさを調整
    # size : 点の大きさ [m]
    # 引数が指定されない場合は，適当にzoom_levelから補間します．
    def set_point_size(self, size = None):
        if size is None:
            self.point_size = 100*10**((12-self.zoom_lv)/3) # zooml_lv 12で100くらい，zoom_lv 15で10くらいが丁度いいので，その比率で適当に補間．
        else:
            self.point_size = size

    # 出力画像の縦と横の大きさを指定 (scaleを変更しているのでこの通りの画素数ではない)
    def set_figure_size(self, height = 450, width = 600):
        self.fig_height = height; self.fig_width = width

    # 射場に関するcsvファイルを読み取る
    # file_names : ファイル名 (パス)
    def read_launch_site_data(self, file_names):
        temp_dfs = [df for df in map(pd.read_csv, file_names)
            if {"longitude", "latitude"} <= set(df.columns) or {"経度", "緯度"} <= set(df.columns) ] # DataFrame読み取り
        for df, name in zip(temp_dfs, file_names):
            df["name"] = Path(name).stem # ファイル名を新たに追加.
            df.rename(columns={ "経度" : "longitude", "緯度" : "latitude" }, inplace=True) # "longitude", "latitude"に統一
        self.launch_site_dfs += temp_dfs # DataFrame追加
        self.launch_site_centroid = gpd.GeoDataFrame(crs="epsg:6668",
            geometry=[Point(df[["longitude", "latitude"]]) if len(df) == 1 else LineString(df[["longitude", "latitude"]]) if len(df) < 3 else Polygon(df[["longitude", "latitude"]])
                for df in self.launch_site_dfs] # 各領域をジオメトリに変換
            ).dissolve().centroid[0] # ジオメトリを統一して, 射場の重心再計算
        # 煩雑なのでUTM座標系に変換していない. 正確じゃないと警告されるがガチガチに求めたいわけじゃないから適当でおk.


    # geopandas.GeoSeries.bufferによって点を囲む領域に変換
    # pointは中心座標(shapely.geometry.Point型), それ以外は引数はgeopandas.GeoSeries.bufferを準用
    # 戻り値はx, y座標のタプル
    def __point_to_buffer(point, distance, resolution, **kwargs):
        gdf = gpd.GeoDataFrame(crs="epsg:6668", geometry=[point])
        utm = gdf.estimate_utm_crs()
        gdf = gdf.to_crs(utm)
        gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(distance, resolution,** kwargs)).to_crs("epsg:6668")
        return gdf["geometry"][0].exterior.xy # ジオメトリは1個のみのはず

    # ポイントを囲む円領域に変換
    # pont : 中心座標 (shapely.geometry.Point型),  radius : 半径
    def __point_to_circle(point, radius, show_center = True):
        resolution = 16 #default
        x, y = scatter_map.__point_to_buffer(point, radius, resolution)
        if not show_center: return list(center_x) + ["none"] + list(x), list(center_y) + ["none"] + list(x)
        center_x, center_y = point.xy
        return list(center_x) + ["none"] + list(x), list(center_y) + ["none"] + list(y)

    # ポイントを囲むX領域に変換
    # pont : 中心座標 (shapely.geometry.Point型),  radius : 半径
    def __point_to_cross(point, size):
        resolution = 1
        x, y = scatter_map.__point_to_buffer(point, size, resolution, cap_style=3)
        x = np.array(x); y = np.array(y)
        y_p_x =  y + x; y_m_x = y - x # y + xと y - x を求める
        lons = [x[np.argmax(y_p_x)], x[np.argmin(y_p_x)], "none", x[np.argmax(y_m_x)], x[np.argmin(y_m_x)]]
        lats = [y[np.argmax(y_p_x)], y[np.argmin(y_p_x)], "none", y[np.argmax(y_m_x)], y[np.argmin(y_m_x)]]
        return lons, lats

    def __draw_lauch_site(self):
        if self.launch_site_dfs is None: return
        for df in self.launch_site_dfs:
            if len(df) == 1: # 点
                lon, lat = scatter_map.__point_to_circle(Point(df[["longitude", "latitude"]]), radius = self.point_size) # 点の大きさ
                self.fig.add_trace(go.Scattermapbox(
                    mode = "markers+lines",
                    lon = lon, lat = lat,
                    line = {"width" : 1}, # 線幅
                    marker = {"size" : [3] + [0 for i in range(len(lat)-1)]}, # 先頭の値は円の中心の点の大きさ
                    fill = "toself", name  = df.at[0, "name"]  # 塗りつぶしなしならfillの値を"none"に
                ))
            elif len(df) < 3:
                self.fig.add_trace(go.Scattermapbox(
                    mode = "lines",
                    lon = df["longitude"].to_list() + [df.at[0, "longitude"]],
                    lat = df["latitude"].to_list() + [df.at[0, "latitude"]],
                    line = {"width" : 1, "color":"black"}, # 線幅
                    fill = "toself", name  = df.at[0, "name"]
                ))
            else: # ポリゴン
                self.fig.add_trace(go.Scattermapbox(
                    mode = "lines",
                    lon = df["longitude"].to_list() + [df.at[0, "longitude"]],
                    lat = df["latitude"].to_list() + [df.at[0, "latitude"]],
                    fill = "toself", name  = df.at[0, "name"]
                ))
    def __set_figure_layout(self, center):
        lon, lat = center.xy
        # 出力するマップの諸元
        self.fig.update_layout(
            mapbox = {
                'center': {'lon' : lon[0], 'lat' : lat[0]},
                'style': "white-bg", 'zoom': self.zoom_lv # ズーム率 海 : 12
            },
            mapbox_layers=[
                {
                    "below": 'traces',"sourcetype": "raster",
                    "source": ["https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png"] # 地理院タイル
                }
            ]
        )
        # 凡例
        self.fig.update_layout(legend={"xanchor":'left', "yanchor":'bottom',"x":0.02, "y":0.9, "orientation":'h'})
        # 余白なし
        self.fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        # アノテーション
        self.fig.add_annotation(
            x=0.99, y=0.01,
            text="地理院タイルを加工して作成", font_color="black", font_size=12,
            showarrow = False, bgcolor="lightgrey"
        )

    # DataFrameの中から経度, 緯度 or longitude, latitudeを探し, 対応するペアを1組ずつ返す.
    def __read_scatter_data_pair(df):
        df.rename(columns=lambda s : s.replace("経度", "longitude").replace("緯度", "latitude")) # "longitude", "latitude"に統一
        for col in df.columns:
            if "longitude" in col and col.replace("longitude", "latitude") in df.columns:
                yield df[[col, col.replace("longitude", "latitude")]] # longitude, latitudeの順で格納して返す

    # 落下分散のDataFrameと風速, 風向, 経度, 緯度に対応する列名
    def draw_scatter_map(self, file_names, col_wind_speed = None, col_wind_dir = None, do_separate = False, show_map = True, DL_img = True):
        if not len(file_names): return # ファイル入力がない場合
        # 出力先ディレクトリ作成
        output_dir = Path(f"Scatter_Map_Results_{int(time.time())}")
        os.mkdir(output_dir)
        for df, name in zip(map(pd.read_csv, file_names), map(Path, file_names)): # DataFrame読み取り
            os.mkdir(output_dir/name.stem) # ファイル名でディレクトリ作成
            for pair in scatter_map.__read_scatter_data_pair(df): # 経度・緯度のペアを1組ずつ読み取り
                self.fig = go.Figure(layout=go.Layout(height=self.fig_height, width=self.fig_width)) # 初期化
                self.__draw_lauch_site() # 射場描画
                if len(df) == 1: # Detail (点) の場合
                    lon, lat = scatter_map.__point_to_cross(Point(pair), size = self.point_size) # Xに変換
                    self.fig.add_trace(go.Scattermapbox(
                        mode = "lines",
                        lon = lon, lat = lat,
                        line = {"width" : 2, "color" : "red"}, # 線幅と色
                        showlegend=False
                    ))
                    scatter_centroid = Point(pair) # 落下分散重心
                else:
                    scatter_lon = []; scatter_lat = []
                    # 風速毎に抽出
                    for wind_speed in set(df.loc[:, col_wind_speed]):
                        df_temp = pair[df[col_wind_speed] == wind_speed]
                        # df_temp = df_temp.sort_values(col_wind_dir) # 風向でソート
                        df_temp = pd.concat([df_temp, df_temp[0:1]])
                        scatter_lon += df_temp.iloc[:, 0].to_list()+ ["none"]
                        scatter_lat += df_temp.iloc[:, 1].to_list() +  ["none"]
                    self.fig.add_trace(go.Scattermapbox(
                        mode = "lines",
                        lon = scatter_lon, lat = scatter_lat,
                        showlegend=False,
                        line= {"color" : "black", "width" : 1} # 色指定
                    ))
                    scatter_centroid = Polygon(pair) # 落下分散重心
                self.__set_figure_layout(gpd.GeoDataFrame(crs="epsg:6668", geometry=[self.launch_site_centroid, scatter_centroid])
                    .dissolve().centroid[0]) # 射場重心と落下分散重心の重心 (中点) をマップの中心に
                if show_map: self.fig.show()
                if DL_img:
                    img_name = pair.columns[0].replace("longitude", "") + ".png"
                    if img_name == ".png":
                        img_name = "output.png"
                    self.fig.write_image(output_dir/name.stem/img_name, engine="kaleido", scale=2)
        # DL用にZIPファイルに圧縮
        if DL_img: shutil.make_archive(output_dir,  format='zip', root_dir=output_dir)
        shutil.rmtree(output_dir) # ディレクトリごと削除
        if DL_img: files.download(output_dir.with_suffix(".zip")) # ファイルダウンロード

## メインコード

In [ ]:
import glob # ファイル構造
import warnings # 警告無視用（任意）
warnings.simplefilter("ignore", UserWarning) # 特定の警告を無視

def main():
    # アップロードするデータの風速，風向，緯度，経度の列名
    col_wind_speed = "wind_speed[m/s]"
    col_wind_dir   = "wind_dir[deg]"

    map = scatter_map()
    map.set_zoom_level(15) # NSE陸
    map.set_point_size()
    # map.set_zoom_level(12.2) # NSE海

    # map.read_launch_site_data(["警戒区域.csv", "保安範囲1.csv", "保安範囲2.csv", "500m警戒線.csv", "射点.csv"])
    map.read_launch_site_data(["落下可能域.csv", "射点.csv"])

    # ファイルアップロード
    file_names = list(files.upload().keys()) # 複数ファイルをいっきにUPしてもOK
    map.draw_scatter_map(file_names, col_wind_speed, col_wind_dir, show_map=True, DL_img=False)
    for f in file_names: os.remove(f) # 射場データ以外のcsvファイルは削除

if __name__ == "__main__":
    main()

## 任意の点周り半径 r の座標をcsvファイルで出力

In [ ]:
import geopandas as gpd     # 緯度・経度処理
from shapely.geometry import Point, Polygon # gpdの座標
import pandas as pd         # csvファイル
from pathlib import Path    # パス
from google.colab import files # ファイルアップロード, ダウンロード
import shutil               # ディレクトリ削除
from pathlib import Path    # パス

latitude = float(input("circle's center latitude : "))
longitude = float(input("circle's center longitude : "))
radius = float(input("circle's radius [m]: "))

# tenpフォルダ作成
output_dir = Path("temp")
if not output_dir.exists():
    os.mkdir(output_dir)

output_file = output_dir / f"{radius}m_circle_at({latitude}_{longitude}).csv"

coord = Point(longitude, latitude) # 基準点 (経度, 緯度の順に注意)
gdf = gpd.GeoDataFrame(crs="epsg:4326", geometry=[coord, coord]) # GeoDataFrame (pd.DataFrameに相当) 作成, 参照座標系はepsg:4326.
utm = gdf.estimate_utm_crs() # UTMを推定
gdf = gdf.to_crs(utm) # UTMに座標変換
gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(radius, 32)).to_crs("epsg:4326") # UTM座標系で距離 {radius} mの円領域を作成し, epsg:4326に戻す
temp = gdf["geometry"][0]  # GeoDataFrameのジオメトリ (この場合は shapely.geometry.Polygon オブジェクト) 取得
temp_x, temp_y = temp.exterior.xy # exterierから外周部のLinerRingオブジェクトを取得, xyによって座標に変換して取得

df = pd.DataFrame({"longitude" : temp_x, "latitude" : temp_y})
df.to_csv(output_file, index=False)
files.download(output_file) # ファイルダウンロード

## 下は試しに作成したコード（使わない）
全体的な流れは分かりやすいので一応残す

In [ ]:
import plotly.express as px       # 地図
import plotly.graph_objects as go # 地図
import geopandas as gpd     # 緯度・経度処理
from shapely.geometry import Point, Polygon # gpdの座標
import pandas as pd         # csvファイル
from pathlib import Path    # パス
import shutil               # ディレクトリ削除
import glob                 # ファイル構造
import os
from google.colab import files # ファイルアップロード, ダウンロード


s_map = scatter_map()
s_map.read_launch_site_data(glob.glob("*.csv"))

### ユーザー入力用諸変数 ###

# アップロードするデータの風速，風向，緯度，経度の列名
col_wind_speed = "wind_speed[m/s]"
col_wind_dir   = "wind_dir[deg]"
col_latitude  = "body1_final_latitude" # 緯度
col_longitude = "body1_final_longitude"# 経度

# ファイルアップロード
upload = files.upload()
file_name = list(upload.keys())[0]
df = pd.read_csv(file_name, encoding="shift-jis") # 文字化けるようならここをutf-8などに変更
df = df.loc[:, [col_wind_speed, col_wind_dir, col_latitude, col_longitude]] # 必要な列を抽出
os.remove(file_name)

# ダウンロード用に一時的に保存するディレクトリ
temp_dir = Path("temp")
if temp_dir.exists():
    shutil.rmtree(temp_dir) # 存在するならディレクトリごと削除
temp_dir.mkdir() # ディレクトリ作成

# 射場の情報が書かれたファイル名
file_name = ["保安範囲", "借用範囲", "本部", "射点", "点火点"]


fig = go.Figure(layout=go.Layout(height=450, width=600))

# ポイントを円形に変換する.
# 引数 :
#  point : shapely.geometry.Point型
#  radius : 半径 [m]
# 戻り値 :
#  中心の点と各円周上の点のxy座標がそれぞれリストとして返される
#  中心の点と円周上の点の間には"none"要素が挿入してある
def point_to_circle(point, radius = 20):
    gdf = gpd.GeoDataFrame(crs="epsg:6668", geometry=[point])
    utm = gdf.estimate_utm_crs()
    gdf = gdf.to_crs(utm)
    gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(radius)).to_crs("epsg:6668")
    center_x, center_y = point.xy
    temp_x, temp_y = gdf["geometry"][0].exterior.xy # ジオメトリは1個のみのはず
    return list(center_x) + ["none"] + list(temp_x), list(center_y) + ["none"] + list(temp_y)

geometries = []

for p in glob.glob('*.csv'):
    file_path = Path(p)
    temp_df = pd.read_csv(file_path)
    temp_df.rename(columns={"緯度" : "latitude", "経度" : "longitude"}, inplace=True) # latitude, longitudeに統一
    if len(temp_df) == 1: # 点
        geometries += [Point(temp_df[["longitude", "latitude"]])]
        lon, lat = point_to_circle(Point(temp_df[["longitude", "latitude"]]), radius = 10)
        fig.add_trace(go.Scattermapbox(
            mode = "lines",
            lon = lon, lat = lat,
            line = {"width" : 0.5}, # 線幅
            marker = {"size" : [3] + [0 for i in range(len(lat)-1)]}, # 先頭の値は円の中心の点の大きさ
            fill = "toself", name  = file_path.stem
        ))
    else: # ポリゴン
        geometries += [Polygon(temp_df[["longitude", "latitude"]])]
        fig.add_trace(go.Scattermapbox(
            mode = "lines",
            lon = temp_df["longitude"].to_list() + [temp_df.at[0, "longitude"]],
            lat = temp_df["latitude"].to_list() + [temp_df.at[0, "latitude"]],
            fill = "toself", name  = file_path.stem
        ))
print(gpd.GeoDataFrame(crs="epsg:6668", geometry=geometries).dissolve().centroid) # centroid of luanch site

scatter_lon = []; scatter_lat = []
# 風速毎に抽出
for wind_speed in set(df.loc[:, col_wind_speed]):
    df_temp = df[df[col_wind_speed] == wind_speed]
    df_temp = df_temp.sort_values(col_wind_dir) # 風向でソート
    df_temp = pd.concat([df_temp, df_temp[0:1]])
    scatter_lon += df_temp[col_longitude].to_list()+ ["none"]
    scatter_lat += df_temp[col_latitude].to_list() +  ["none"]

fig.add_trace(go.Scattermapbox(
    mode = "lines",
    lon = scatter_lon, lat = scatter_lat,
    name  = "scatter",
    line= {"color" : "black"}
))
# 出力するマップの諸元
fig.update_layout(
    mapbox = {
        'center': {'lat':34.2855, 'lon':135.090582},
        'style': "white-bg",
        'zoom': 14.75 # ズーム率
    },
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png" # 地理院タイル
            ]
        }
    ]
)
# 凡例
fig.update_layout(
    legend=dict(
        xanchor='left',
        yanchor='bottom',
        x=0.02,
        y=0.9,
        orientation='h',
        )
)
# 余白なし
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# アノテーション
fig.add_annotation(
    text="地理院タイルを加工して作成",
    font_color="black",
    font_size=12,
    showarrow = False,
    x=0.99, y=0.01,
    bgcolor="lightgrey"
)
fig.show()

Saving para_without_detachment.csv to para_without_detachment.csv
0    POINT (135.09068 34.28465)
dtype: geometry


<ipython-input-27-03e24ef1bba5>:83: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




## メモ

地図描画用に用いたplotlyの関数ではmapboxというサービスを利用する (さらに裏でjsを呼び出しているのでJSONファイル味が...).

公式のReferenceはこれ : https://plotly.com/python/maps/.
あまり体系的に纏められていないので, 困ったら関数名でググった方が早い.
<!--
mapboxは従量課金制だが, 特定のマップはトークンの発行なしにフリーで使える.

トークンなしに使えるマップスタイルは公式HP (https://plotly.com/python/mapbox-layers/) に纏められていて, 次の3種類 (2024年2月時点) :
 * "open-street-map"
 * "carto-positron"
 * "carto-darkmatter"

調べた限り上記マップの使用に際してライセンス等の問題はないと思われる
(なお, Plotly自体はフリー (https://plotly.com/python/is-plotly-free/)). -->


<!-- また, 公式HP (同上) ではUSGS (アメリカ地質調査所) のマップタイルを用いた例を紹介している (これもフリー). -->

web上の地図タイル (API的なもの) を用いることで地図を表示することができる.

日本版の地図タイルとしては, 地理院タイルなどがある.
地理院タイルの場合は **国土地理院等のライセンス表記が必須**なので注意.
> 地理院タイル : https://maps.gsi.go.jp/development/ichiran.html
>
> 同上 利用規約 : https://www.gsi.go.jp/kikakuchousei/kikakuchousei40182.html

使い勝手とバリエーションの点からは地理院タイルの方を用いた方がよいと思う(ライセンス表記が必要だが...).
****
地球上の座標を扱うには, 扱う座標系 (参照座標系 : CRS) の選択が重要となる
(詳細は https://www.aeroasahi.co.jp/fun/column/19/ などが詳しい).

** **

緯度・経度の座標系は,  "epsg:6668" (日本測地系2011) ないし "epsg:4612" (日本測地系2000) を用いればよい (cf : https://lemulus.me/column/epsg-list-gis) .

しかし, 緯度・経度の情報からでは距離を割り出すのは困難.

そこで, 距離を割り出すためには距離が扱える都合いい座標系 : UTM (cf : https://utmgrid.org/utmgrid-1/) への座標変換が必要.

基準点からの円形を描画するにあたっては, GeoPandasのbuffer関数を用いる (厳密にはshapely.geometry).
この関数はCRSがUTMだと怒られる.

なお, utm座標系は色々あって面倒なので, 適当なutm座標系の取得には .estimate_utm_crs関数を用いる.

以下, テスト用 & GeoPandas解説用プログラム

In [ ]:
import plotly.express as px
import plotly.graph_objects as go # 地図
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Polygon, Point
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot

coord = Point(135.090582, 34.285238) # 適当な基準点 (経度, 緯度の順に注意)
print(type(coord))
gdf = gpd.GeoDataFrame(crs="epsg:6668", geometry=[coord, coord]) # GeoDataFrame (pd.DataFrameに相当) 作成, 参照座標系はepsg:6668.
utm = gdf.estimate_utm_crs() # UTMを推定
gdf = gdf.to_crs(utm) # UTMに座標変換
gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(100)).to_crs("epsg:6668") # UTM座標系で距離100mの円領域を作成し, epsg:6668に戻す
temp = gdf["geometry"][0]  # GeoDataFrameのジオメトリ (この場合は shapely.geometry.Polygon オブジェクト) 取得
temp_x, temp_y = temp.exterior.xy # exterierから外周部のLinerRingオブジェクトを取得, xyによって座標に変換して取得
# メモ : 緯度, 経度の順ではなく, 経度, 緯度の順

fig = go.Figure(layout=go.Layout(height=600, width=800))
fig.add_trace(go.Scattermapbox(
    mode = "markers+lines",
    lat =[34.285238, "none"] + list(temp_y), # 中心点も一緒に描画する, このとき, "none"を挟むとその前後で辺を切断できる
    lon = [135.090582, "none"] + list(temp_x),
    fill = "toself",
    marker = { "size" : [10] + [0 for i in range(len(temp_x) + 1)] }, # 頂点の大きさを0にして非表示に
    name = "center"
))

coord = Point(135.09, 34.29) # 適当な基準点その2
print(type(coord))
gdf = gpd.GeoDataFrame(crs="epsg:6668", geometry=[coord, coord]) # GeoDataFrame (pd.DataFrameに相当) 作成, 参照座標系はepsg:6668.
utm = gdf.estimate_utm_crs() # UTMを推定
gdf = gdf.to_crs(utm) # UTMに座標変換
gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(10, 1, cap_style=3)).to_crs("epsg:6668") # UTM座標系で距離100mの正方形領域を作成し, epsg:6668に戻す
temp = gdf["geometry"][0]  # GeoDataFrameのジオメトリ (この場合は shapely.geometry.Polygon オブジェクト) 取得
temp_x, temp_y = temp.exterior.xy # exterierから外周部のLinerRingオブジェクトを取得, xyによって座標に変換して取得
temp_x = np.array(temp_x); temp_y = np.array(temp_y)
temp_y_p_x =  temp_y + temp_x; temp_y_m_x = temp_y - temp_x
lons = [temp_x[np.argmax(temp_y_p_x)], temp_x[np.argmin(temp_y_p_x)], "none", temp_x[np.argmax(temp_y_m_x)], temp_x[np.argmin(temp_y_m_x)]]
lats = [temp_y[np.argmax(temp_y_p_x)], temp_y[np.argmin(temp_y_p_x)], "none", temp_y[np.argmax(temp_y_m_x)], temp_y[np.argmin(temp_y_m_x)]]

fig.add_trace(go.Scattermapbox(
    mode = "lines",
    lon = lons,
    lat = lats,
    fill = "toself",
    name = "rocket",
    line = {"color" : "red"},
    showlegend=False
))
fig.update_layout(
    mapbox = {
        #'accesstoken': # WRITE ACCES TOKEN IF YOU NEED ##
        'center': {'lat':34.285238, 'lon':135.090582},
        'zoom': 15
    },
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png" # 地理院タイル
            ]
        }
    ],
    legend=dict(
        xanchor='left',
        yanchor='bottom',
        x=0.02,
        y=0.9,
        orientation='h',
        ),
    margin={"r":0,"t":0,"l":0,"b":0} # 余白なし
)
fig.show()

<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>


In [ ]:
import plotly.express as px
import plotly.graph_objects as go # 地図
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Polygon, Point
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot

fig = go.Figure(layout=go.Layout(height=600, width=800))

coords = [Point(135.0903599, 34.2858134), Point(135.09058057, 34.2851168)]
for coord in coords:
    print(type(coord))
    gdf = gpd.GeoDataFrame(crs="epsg:6668", geometry=[coord, coord]) # GeoDataFrame (pd.DataFrameに相当) 作成, 参照座標系はepsg:6668.
    utm = gdf.estimate_utm_crs() # UTMを推定
    gdf = gdf.to_crs(utm) # UTMに座標変換
    gdf = gpd.GeoDataFrame(crs=utm, geometry=gdf.buffer(10, 1, cap_style=3)).to_crs("epsg:6668") # UTM座標系で距離100mの正方形領域を作成し, epsg:6668に戻す
    temp = gdf["geometry"][0]  # GeoDataFrameのジオメトリ (この場合は shapely.geometry.Polygon オブジェクト) 取得
    temp_x, temp_y = temp.exterior.xy # exterierから外周部のLinerRingオブジェクトを取得, xyによって座標に変換して取得
    temp_x = np.array(temp_x); temp_y = np.array(temp_y)
    temp_y_p_x =  temp_y + temp_x; temp_y_m_x = temp_y - temp_x
    lons = [temp_x[np.argmax(temp_y_p_x)], temp_x[np.argmin(temp_y_p_x)], "none", temp_x[np.argmax(temp_y_m_x)], temp_x[np.argmin(temp_y_m_x)]]
    lats = [temp_y[np.argmax(temp_y_p_x)], temp_y[np.argmin(temp_y_p_x)], "none", temp_y[np.argmax(temp_y_m_x)], temp_y[np.argmin(temp_y_m_x)]]

    fig.add_trace(go.Scattermapbox(
        mode = "lines",
        lon = lons,
        lat = lats,
        fill = "toself",
        name = "rocket",
        line = {"color" : "red"},
        showlegend=False
    ))

fig.update_layout(
    mapbox = {
        #'accesstoken': # WRITE ACCES TOKEN IF YOU NEED ##
        'center': {'lat':34.285238, 'lon':135.090582},
        'zoom': 15
    },
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "source": [
                "https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png" # 地理院タイル
            ]
        }
    ],
    legend=dict(
        xanchor='left',
        yanchor='bottom',
        x=0.02,
        y=0.9,
        orientation='h',
        ),
    margin={"r":0,"t":0,"l":0,"b":0} # 余白なし
)
fig.show()

<class 'shapely.geometry.point.Point'>
<class 'shapely.geometry.point.Point'>


# 作成者
佐藤空馬 (14期)

mail : sato.kuma.t7(at)dc.tohoku.ac.jp